In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import sqlite3
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
sales = pd.read_csv('data/sales.csv')

In [3]:
sales.columns

Index(['Unnamed: 0', 'id', 'quantity', 'name', 'price', 'fs_receipt_id',
       'fs_receipt_issue_date', 'org_id', 'org_ico', 'org_dic',
       'org_building_number', 'org_country', 'org_ic_dph', 'org_municipality',
       'org_postal_code', 'org_name', 'org_street_name', 'unit_id',
       'unit_building_number', 'unit_country', 'unit_municipality',
       'unit_postal_code', 'unit_property_registration_number',
       'unit_street_name', 'unit_name', 'ai_name_without_brand_and_quantity',
       'ai_name_in_english_without_brand_and_quantity', 'ai_brand',
       'ai_category', 'ai_quantity_value', 'ai_quantity_unit', 'unit_latitude',
       'unit_longitude'],
      dtype='object')

In [4]:
sales.isna().sum()

Unnamed: 0                                          0
id                                                  0
quantity                                            0
name                                                0
price                                               0
fs_receipt_id                                       0
fs_receipt_issue_date                               0
org_id                                              0
org_ico                                             0
org_dic                                             0
org_building_number                              6857
org_country                                         0
org_ic_dph                                         71
org_municipality                                    0
org_postal_code                                  8434
org_name                                            0
org_street_name                                     0
unit_id                                             0
unit_building_number        

In [5]:
def clean_org_name(name: str) -> str:
    if pd.isna(name):
        return name
    name = re.sub(
        r'\b(s\s*\.?\s*r\s*\.?\s*o\.?|a\s*\.?\s*s\.?|v\s*\.?\s*o\s*\.?\s*s\.?|spol\s*\.?\s*s\s*r\s*\.?\s*o\.?)\b',
        '',
        name,
        flags=re.IGNORECASE
    )
    name = re.sub(r'\b(Slovenská republika|Slovensko|SR|Slovakia|SK)\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'[.,/]', ' ', name)
    name = re.sub(r'\s{2,}', ' ', name)
    name = name.strip(" -_")
    if 'COOP' in name.upper():
        name = 'COOP Jednota'
    return name

In [6]:
sales.drop('Unnamed: 0', axis=1, inplace=True)
sales = sales.rename(columns={'fs_receipt_id':'receipt_id', 'fs_receipt_issue_date': 'issue_date', 'org_country': 'country',
                              'unit_municipality': 'city', 'unit_street_name': 'street', 'ai_name_without_brand_and_quantity': 'product_name',
                              'ai_name_in_english_without_brand_and_quantity': 'product_name_en', 'ai_brand': 'brand', 'ai_category': 'category',
                              'ai_quantity_value': 'quantity_value', 'ai_quantity_unit': 'quantity_unit', 'unit_latitude': 'latitude', 'unit_longitude': 'longitude'
                              })
sales.loc[sales['city'].isna(), 'city'] = (sales['org_municipality'].str.split('-').str[0].str.strip())
sales.loc[sales['street'].isna(), 'street'] = (sales['org_street_name'].str.split('-').str[0].str.strip())
sales["issue_date"] = pd.to_datetime(sales["issue_date"],errors="coerce").dt.tz_localize(None)
sales['month'] = sales['issue_date'].dt.month
sales['year'] = sales['issue_date'].dt.year
sales['month_year'] = sales['issue_date'].dt.to_period('M').astype(str)
sales['weekday'] = sales['issue_date'].dt.day_name()
sales['receipt_price'] = sales['price'] * sales['quantity']
sales['is_luxury'] = np.where(sales['price'] > 50, 1, 0)
sales['is_bio'] = np.where(sales['name'].str.contains('bio', na=False, case=False), 1, 0)
sales['is_eco'] = np.where(sales['name'].str.contains('eco', na=False, case=False), 1, 0)
sales['org_name'] = sales['org_name'].apply(clean_org_name)
sales.loc[sales['city'].str.contains('Bratislava', na=False, case=False), 'city'] = 'Bratislava'
sales.loc[sales['city'].str.contains('Šaštín', na=False, case=False), 'city'] = 'Šaštín-Stráže'
sales['city'] = sales['city'].str.split('-').str[0].str.strip()

In [7]:
for col in sales.select_dtypes(include=['object']).columns:
    sales[col] = sales[col].fillna("Unknown")
for col in sales.select_dtypes(include=['number']).columns:
    sales[col] = sales[col].fillna(0)

In [8]:
import sqlite3
conn = sqlite3.connect('sales.db')
sales.to_sql('sales', conn, if_exists='replace', index=False)
conn.close()

In [9]:
sales.columns

Index(['id', 'quantity', 'name', 'price', 'receipt_id', 'issue_date', 'org_id',
       'org_ico', 'org_dic', 'org_building_number', 'country', 'org_ic_dph',
       'org_municipality', 'org_postal_code', 'org_name', 'org_street_name',
       'unit_id', 'unit_building_number', 'unit_country', 'city',
       'unit_postal_code', 'unit_property_registration_number', 'street',
       'unit_name', 'product_name', 'product_name_en', 'brand', 'category',
       'quantity_value', 'quantity_unit', 'latitude', 'longitude', 'month',
       'year', 'month_year', 'weekday', 'receipt_price', 'is_luxury', 'is_bio',
       'is_eco'],
      dtype='object')

In [10]:
sales['city'].value_counts().head(10)

city
Bratislava          6793
Šaštín               519
Štefanov             381
Senica               249
Kúty                 182
Stupava               42
Košice                38
Slovenský Grob        31
Banská Štiavnica      27
Jelka                 23
Name: count, dtype: int64

In [11]:
sales.columns

Index(['id', 'quantity', 'name', 'price', 'receipt_id', 'issue_date', 'org_id',
       'org_ico', 'org_dic', 'org_building_number', 'country', 'org_ic_dph',
       'org_municipality', 'org_postal_code', 'org_name', 'org_street_name',
       'unit_id', 'unit_building_number', 'unit_country', 'city',
       'unit_postal_code', 'unit_property_registration_number', 'street',
       'unit_name', 'product_name', 'product_name_en', 'brand', 'category',
       'quantity_value', 'quantity_unit', 'latitude', 'longitude', 'month',
       'year', 'month_year', 'weekday', 'receipt_price', 'is_luxury', 'is_bio',
       'is_eco'],
      dtype='object')

In [12]:
sales[sales['name'] == '5904862542003 OBUV CP40-SC2116']

,id,quantity,name,price,receipt_id,issue_date,org_id,org_ico,org_dic,org_building_number,country,org_ic_dph,org_municipality,org_postal_code,org_name,org_street_name,unit_id,unit_building_number,unit_country,city,unit_postal_code,unit_property_registration_number,street,unit_name,product_name,product_name_en,brand,category,quantity_value,quantity_unit,latitude,longitude,month,year,month_year,weekday,receipt_price,is_luxury,is_bio,is_eco
66,2153,1.0,5904862542003 OBUV CP40-SC2116,22.99,193,2023-06-27 17:37:23,24,46509500,2023414492,Unknown,Slovensko,SK2023414492,Bratislava - mestská časť Ružinov,0.0,CCC,Ivanská cesta,28,Unknown,Slovensko,Bratislava,82108.0,6,Metodova,Unknown,Unknown,Unknown,Unknown,Auto Care,0.0,Unknown,48.15702,17.129524,6,2023,2023-06,Tuesday,22.99,0,0,0


In [13]:
sales.head(5)

,id,quantity,name,price,receipt_id,issue_date,org_id,org_ico,org_dic,org_building_number,country,org_ic_dph,org_municipality,org_postal_code,org_name,org_street_name,unit_id,unit_building_number,unit_country,city,unit_postal_code,unit_property_registration_number,street,unit_name,product_name,product_name_en,brand,category,quantity_value,quantity_unit,latitude,longitude,month,year,month_year,weekday,receipt_price,is_luxury,is_bio,is_eco
0,1212,1.0,4-71-12-0641-8 30.0,29.95,71,2022-01-18 16:12:53,28,35832932,2020215912,Unknown,Slovensko,SK2020215912,Bratislava - mestská časť Petržalka,0.0,Leder & Schuh,Einsteinova,33,Unknown,Slovensko,Bratislava,82109.0,16,Mlynské Nivy,Unknown,Unknown,Unknown,Unknown,Vitamins & Ointments,0.0,Unknown,48.145888,17.127227,1,2022,2022-01,Tuesday,29.95,0,0,0
1,1211,1.0,4-71-12-0695-8 33.0,37.95,71,2022-01-18 16:12:53,28,35832932,2020215912,Unknown,Slovensko,SK2020215912,Bratislava - mestská časť Petržalka,0.0,Leder & Schuh,Einsteinova,33,Unknown,Slovensko,Bratislava,82109.0,16,Mlynské Nivy,Unknown,Unknown,Unknown,Unknown,Baking,0.0,Unknown,48.145888,17.127227,1,2022,2022-01,Tuesday,37.95,0,0,0
2,1213,1.0,5904248240332 OBUV CI12-JOY-03(III)CH,39.99,72,2022-03-24 16:06:02,24,46509500,2023414492,Unknown,Slovensko,SK2023414492,Bratislava - mestská časť Ružinov,0.0,CCC,Ivanská cesta,28,Unknown,Slovensko,Bratislava,82108.0,6,Metodova,Unknown,Obuv CI12-JOY-03(III)CH,Footwear CI12-JOY-03(III)CH,Unknown,Footwear,0.0,Unknown,48.157020,17.129524,3,2022,2022-03,Thursday,39.99,0,0,0
3,1214,1.0,5900949525822 PRÍSLUŠENSTVO,2.95,72,2022-03-24 16:06:02,24,46509500,2023414492,Unknown,Slovensko,SK2023414492,Bratislava - mestská časť Ružinov,0.0,CCC,Ivanská cesta,28,Unknown,Slovensko,Bratislava,82108.0,6,Metodova,Unknown,Accessory,Unknown,Unknown,Unknown,0.0,Unknown,48.157020,17.129524,3,2022,2022-03,Thursday,2.95,0,0,0
4,1148,1.0,5904248405014 OBUV CI12-HARRY-01,39.99,59,2022-05-06 18:21:47,24,46509500,2023414492,Unknown,Slovensko,SK2023414492,Bratislava - mestská časť Ružinov,0.0,CCC,Ivanská cesta,28,Unknown,Slovensko,Bratislava,82108.0,6,Metodova,Unknown,Obuv,Footwear,Unknown,Accessories,0.0,Unknown,48.157020,17.129524,5,2022,2022-05,Friday,39.99,0,0,0


In [ ]:
top_city_by_rows = (
    sales.dropna(subset=['city'])
         .groupby('city')
         .size()
         .sort_values(ascending=False)
         .index[0]
)


city
Bratislava    6793
Name: count, dtype: int64
